In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,47328
2,Huelva,Confirmados PDIA 14 días,757
3,Huelva,Tasa PDIA 14 días,"147,51446888945185"
4,Huelva,Confirmados PDIA 7 días,404
5,Huelva,Tasa PDIA 7 dias,"78,7263479938422"
6,Huelva,Total Confirmados,47531
7,Huelva,Curados,45733
8,Huelva,Fallecidos,423


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  47328.0


/tmp/ipykernel_101893/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13113.0


/tmp/ipykernel_101893/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_101893/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_101893/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_101893/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 946 personas en los últimos 7 días 

Un positivo PCR cada 465 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,47328.0,404.0,757.0,513170.0,78.726348,147.514469,100.0
Huelva-Costa,27986.0,234.0,462.0,289548.0,80.815616,159.559037,57.0
Huelva (capital),13113.0,152.0,309.0,143837.0,105.675174,214.826505,40.0
Condado-Campiña,14750.0,139.0,251.0,156231.0,88.970819,160.659536,34.0
Palos de la Frontera,1172.0,30.0,61.0,11742.0,255.493102,519.502640,10.0
Sierra de Huelva-Andévalo Central,4164.0,31.0,43.0,67391.0,46.000208,63.806740,9.0
Bollullos Par del Condado,1350.0,17.0,30.0,14387.0,118.162230,208.521582,9.0
Cartaya,2241.0,33.0,74.0,20083.0,164.318080,368.470846,6.0
Almonte,2371.0,9.0,20.0,24507.0,36.724201,81.609336,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granada de Río-Tinto (La),14.0,7.0,9.0,243.0,2880.658436,3703.703704,1.0
Paterna del Campo,232.0,14.0,18.0,3457.0,404.975412,520.682673,2.0
Palos de la Frontera,1172.0,30.0,61.0,11742.0,255.493102,519.502640,10.0
Cartaya,2241.0,33.0,74.0,20083.0,164.318080,368.470846,6.0
Alosno,248.0,12.0,14.0,3933.0,305.110603,355.962370,1.0
Moguer,2108.0,36.0,57.0,21867.0,164.631637,260.666758,4.0
Cañaveral de León,34.0,1.0,1.0,396.0,252.525253,252.525253,NaN
Palma del Condado (La),1468.0,13.0,24.0,10801.0,120.359226,222.201648,1.0
Huelva (capital),13113.0,152.0,309.0,143837.0,105.675174,214.826505,40.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
San Bartolomé de la Torre,398.0,1.0,8.0,3761.0,26.588673,212.709386,0.0,0.125000
Isla Cristina,3196.0,2.0,11.0,21393.0,9.348852,51.418688,1.0,0.181818
Bonares,473.0,1.0,5.0,6060.0,16.501650,82.508251,0.0,0.200000
Cartaya,2241.0,33.0,74.0,20083.0,164.318080,368.470846,6.0,0.445946
Almonte,2371.0,9.0,20.0,24507.0,36.724201,81.609336,5.0,0.450000
Gibraleón,967.0,5.0,11.0,12737.0,39.255712,86.362566,0.0,0.454545
Palos de la Frontera,1172.0,30.0,61.0,11742.0,255.493102,519.502640,10.0,0.491803
Huelva (capital),13113.0,152.0,309.0,143837.0,105.675174,214.826505,40.0,0.491909
Trigueros,552.0,3.0,6.0,7862.0,38.158229,76.316459,1.0,0.500000
